# Running forward dynamics in a connectome-constrained model

The goal of this tutorial is to walk you through the following steps:

- Initializing a sparse neural network model based on a connectome. In this example, we will use the optic lobe of *Drosophila* as done in [Lappalainen et al. (2024)](https://www.nature.com/articles/s41586-024-07939-3).
- Specifying input and output projections.
- Driving activity in the network with visual inputs.

For demonstration purposes, we'll use flattened MNIST images as inputs into the connectome. This is however simplistic and we do allow for arbitrarily complex input mappings. To learn how to do that please refer to the API.

In [1]:
import torch
import os
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

from bioplnn.models import ConnectomeODERNN

Check the device to make sure you are on a GPU. If you aren't its not a big deal. Its just going to take much longer!

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_float32_matmul_precision("high")
print("Using device: {}".format(device))

Download the connectome and read it in as a torch tensor. We have pre-processed this as a sparse tensor for the purposes of this example.

In [ ]:
save_dir = "connectivity/turaga"
os.makedirs(save_dir, exist_ok=True)
save_path = f"{save_dir}/turaga-dros-visual-connectome.pt"
!gdown "https://drive.google.com/uc?id=18448HYpYrm60boziHG73bxN4CK5jG-1g" -O "{save_path}" 
connectome = torch.load(save_path, weights_only=True)

In [ ]:
print(
    "Connectome dimensions: {}x{}".format(
        connectome.shape[0], connectome.shape[1]
    )
)
print("Number of synapses: {}".format(connectome._nnz()))

spatial_extent = 100
vmin, vmax = connectome.values().min(), connectome.values().max()
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
ax.imshow(
    torch.abs(connectome.to_dense()[:spatial_extent, :spatial_extent]),
    cmap="inferno",
)
ax.set_xlabel("Postsynaptic", fontsize=18)
ax.set_ylabel("Presynaptic", fontsize=18)
ax.set_xticks([0, spatial_extent - 1])
ax.set_yticks([0, spatial_extent - 1])
plt.show()


## Creating input and output projection matrices
To drive the network with external inputs, you'd want to specify the subset of neurons in the model that receive input or project to downstream areas. We have an utility to create these sparse projection matrices. For the purposes of this example, we shall pick a random subset of input/output neurons. 
In a world where each neuron receives external input, you can also initialize this projection as an arbitrary *dense* matrix.

In [23]:
from bioplnn.utils.torch import create_identity_ih_connectivity

# since we are feeding in MNIST images
input_size = 28 * 28
num_neurons = connectome.shape[0]

input_projection_matrix = create_identity_ih_connectivity(
    input_size=input_size,
    num_neurons=num_neurons,
    input_indices=torch.randint(high=num_neurons, size=(input_size,)),
)

# for now, lets just read outputs from all neurons
output_projection_matrix = None

## Setting up the connectome-constrained model

In [ ]:
connectome_rnn_kwargs = {
    "input_size": input_size,
    "hidden_size": num_neurons,
    "connectivity_hh": connectome,
    "connectivity_ih": input_projection_matrix,
    "output_neurons": output_projection_matrix,
    "num_neuron_types": 2,
    "neuron_type_class": ["excitatory", "inhibitory"],
    "neuron_type_indices": [
        "connectivity/sunny/indices_neuron_type_1.pt",
        "connectivity/sunny/indices_neuron_type_2.pt",
    ],
    "neuron_type_nonlinearity": "Sigmoid",
    "neuron_type_tau_init": [1.25, 2.0],
    "train_tau": True,
    "batch_first": False,
    "compile_solver_kwargs": {
        "mode": "max-autotune",
        "dynamic": False,
        "fullgraph": True,
    },
}
model = ConnectomeODERNN(**connectome_rnn_kwargs).to(device)
print(model)

In [25]:
# get some data for us to pipe into the model
transform = T.Compose([T.ToTensor(), T.Normalize((0.1307,), (0.3081,))])
train_data = MNIST(root="data", train=True, transform=transform, download=True)
train_loader = DataLoader(
    train_data, batch_size=8, num_workers=0, shuffle=True
)

In [ ]:
# getting one batch of the input
x, label = next(iter(train_loader))
print(f"x shape: {x.shape}, label_shape: {label.shape}")
x = x.flatten(1)
x = x.to(device)
print(f"x flattened shape: {x.shape}")

In [ ]:
model.eval()
_, neural_activities, timesteps = model(
    x, start_time=0, end_time=1.0, num_steps=20
)
print(f"Neural activity shape: {neural_activities.shape}")

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
ax.plot(
    timesteps[:, 0].detach().cpu().numpy(),
    neural_activities[:, 0, torch.randint(0, 47521, (25,))]
    .detach()
    .cpu()
    .numpy(),
)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xlabel("Time (in a.u.)", fontsize=18)
ax.set_ylabel("Activity", fontsize=18)